In [ ]:
#pip install cx_Oracle

In [ ]:
import pandas as pd
import cx_Oracle

In [ ]:
def create_table():   
    with cx_Oracle.connect(user=user, password=password,
                        dsn="some_dsn",
                       encoding="UTF-8") as connection:
        cursor = connection.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS PROCUREMENT_REGISTER
                (
                    ID           NUMBER GENERATED BY DEFAULT AS IDENTITY,
                    ID_DF        VARCHAR2(100),
                    CONTRACT_NUM VARCHAR2(30),
                    OBJECT_NAME  VARCHAR2(2000), 
                    COST         NUMBER, 
                    DESCR        VARCHAR2(2000), 
                    OKPD_NUM     VARCHAR2(20),
                    "COMMENT"    VARCHAR2(4000)
                )''')
        connection.commit()

In [ ]:
create_table()

In [ ]:
def upload_data(filename, chunksize):
    with pd.read_csv(filename, , chunksize=CHUNKSIZE, sep=',', on_bad_lines='skip', encoding='utf-8', low_memory=False) as reader:
                for df in reader:
                    df = df.drop([2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24], axis=1)
                    df = df.rename(columns={0: 'ID',
                                            1: 'CONTRACT_NUM',
                                            11: 'OBJECT_NAME',
                                            21: 'COST',
                                            25: 'DESCR',
                                            26: 'OKPD_NUM'})
                    df['OBJECT_NAME'] = df['OBJECT_NAME'].astype(str)
                    df['OKPD_NUM'] = df['OKPD_NUM'].astype(str)
                    df['COST'] = pd.to_numeric(df['COST'], errors='coerce')
                    df['ID'] = df['ID'].astype(str)
                    df['DESCR'] = df['DESCR'].astype(str)
                    df['CONTRACT_NUM'] = df['CONTRACT_NUM'].astype(str)
    return df

In [ ]:
df_import = upload_data(filename=r'', chunksize=somevalue)

In [ ]:
def load_data_to_db():
    rows = [tuple(x) for x in df_import.values]
    with cx_Oracle.connect(user=user, password=password,
                        dsn="some_dsn",
                       encoding="UTF-8") as connection:
        cur = connection.cursor()
        cur.executemany('INSERT INTO PROCUREMENT_REGISTER(ID_DF, CONTRACT_NUM, OBJECT_NAME, COST, DESCR, OKPD_NUM) VALUES (:1, :2, :3, :4, :5, :6)', rows)
        connection.commit()   

In [ ]:
load_data_to_db(df_import)

In [ ]:
def extract_data_from_db():
    df_drop = pd.DataFrame(columns=['ID_DF', 'CONTRACT_NUM', 'OBJECT_NAME', 'COST', 'DESCR', 'OKPD_NUM'])
    with cx_Oracle.connect(user=user, password=password,
                        dsn="some_dsn",
                       encoding="UTF-8") as connection:
        cur1 = connection.cursor()
        df_drop_db = cur1.execute('SELECT * FROM PROCUREMENT_REGISTER')
        for row in df_drop_db:
            new_row = {'ID_DF': row[0], 'CONTRACT_NUM': row[1], 'OBJECT_NAME': row[2], 'COST': row[3], 'DESCR': row[4], 'OKPD_NUM': row[5]}
            df_drop.append(new_row, ignore_index=True)
    return df_drop

In [ ]:
df_drop = extract_data_from_db()